In [1]:
import psycopg2, os

### Connecting to the database

In [2]:
print("Connecting to the PostgreSQL database...")
conn = psycopg2.connect(
    host = "localhost",
    port = "5432",
    database = "postgres",
    user = "postgres",
    password = "123"
)

Connecting to the PostgreSQL database...


In [3]:
cur = conn.cursor()

### Creating the tables

In [4]:
cur.execute("DROP TABLE departments CASCADE")
cur.execute("DROP TABLE employees CASCADE")
cur.execute("DROP TABLE managers CASCADE")
cur.execute("DROP TABLE customers CASCADE")
cur.execute("DROP TABLE interaction_techniques CASCADE")
cur.execute("DROP TABLE interaction_reasons CASCADE")
cur.execute("DROP TABLE interactions CASCADE")
cur.execute("DROP TABLE shop_categories CASCADE")
cur.execute("DROP TABLE shop_subcategories CASCADE")
cur.execute("DROP TABLE shops CASCADE")
cur.execute("DROP TABLE products CASCADE")
cur.execute("DROP TABLE website_views CASCADE")
cur.execute("DROP TABLE orders CASCADE")
cur.execute("DROP TABLE return_reasons CASCADE")
cur.execute("DROP TABLE return_methods CASCADE")
cur.execute("DROP TABLE returns CASCADE")

In [5]:
createCMD = """ 
CREATE TABLE departments (
	id					varchar(3) NOT NULL,
	department_name		varchar(100),
	annual_budget		numeric(15,2),
	PRIMARY KEY (id)
);

CREATE TABLE employees (
	id				varchar(6) NOT NULL,
	department_id	varchar(3),
	first_name		varchar(50),
	last_name		varchar(50),
	email			varchar(150),
	gender			varchar(30),
	PRIMARY KEY (id),
	FOREIGN KEY (department_id) REFERENCES departments(id)
);

CREATE TABLE managers (
	manager_id		varchar(6) NOT NULL,
	employee_id		varchar(6) NOT NULL,
	department_id	varchar(3),
	PRIMARY KEY (manager_id, employee_id),
	FOREIGN KEY (manager_id) REFERENCES employees(id),
	FOREIGN KEY (employee_id) REFERENCES employees(id),
	FOREIGN KEY (department_id) REFERENCES departments(id)
);

CREATE TABLE customers (
	customer_id		varchar(6) NOT NULL,
	first_name		varchar(50),
	last_name		varchar(50),
	email			varchar(150),
	gender			varchar(30),
	address			varchar(200),
	state			varchar(50),
	PRIMARY KEY (customer_id)
);
 
CREATE TABLE interaction_techniques (
	technique_id 	varchar(2) NOT NULL,
	description 	varchar(50),
	PRIMARY KEY (technique_id)
);

CREATE TABLE interaction_reasons (
	reason_id 		varchar(2) NOT NULL,
	description 	varchar(50),
	PRIMARY KEY (reason_id)
);
 
CREATE TABLE interactions (
	interaction_id 		varchar(6) NOT NULL,
	customer_id 		varchar(6),
	employee_id 		varchar(6),
	technique_id 		varchar(2),
	reason_id 			varchar(2),
	PRIMARY KEY (interaction_id),
	FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
	FOREIGN KEY (employee_id) REFERENCES employees(id),
	FOREIGN KEY (technique_id) REFERENCES interaction_techniques(technique_id),
	FOREIGN KEY (reason_id) REFERENCES interaction_reasons(reason_id)
);

CREATE TABLE shop_categories (
	category_id 			varchar(3) NOT NULL,
	category_description 	varchar(75),
	PRIMARY KEY (category_id)
);

CREATE TABLE shop_subcategories (
	subcategory_id 			varchar(3) NOT NULL,
	subcategory_description varchar(75),
	PRIMARY KEY (subcategory_id)
);

CREATE TABLE shops (
	shop_id					varchar(6) NOT NULL,
	shop_name				varchar(500),
	category_id				varchar(3),
	subcategory_id			varchar(3),
	creation_date			date,
	listings_active_count	int,
	num_favorites			int,
	currency_code			varchar(4),
	is_shop_US_based		varchar(5),
	sale_message			varchar(5),
	sales_count				int,
	review_count			int,
	shop_location			varchar(75),
	PRIMARY KEY (shop_id),
	FOREIGN KEY (category_id) REFERENCES shop_categories(category_id),
	FOREIGN KEY (subcategory_id) REFERENCES shop_subcategories(subcategory_id)
);

CREATE TABLE products (   
	product_id          varchar(6) NOT NULL,
    product_name        varchar(500),
    seller_id           varchar(6),
	rating              numeric(2,1),
	reviews             int,
	price               numeric,
	badget              varchar(200),
	product_url         varchar(1000),
	image_url           varchar(1000),
    PRIMARY KEY(product_id),
	FOREIGN KEY (seller_id) REFERENCES shops(shop_id)
);

CREATE TABLE website_views (
	view_id			varchar(6) NOT NULL,
	product_id		varchar(6),
	view_date		date,
	view_time		timestamp,
	ip_address		varchar(50),
	total_seconds	numeric,
	PRIMARY KEY (view_id),
	FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE orders (
	order_id		varchar(6) NOT NULL,
	customer_id		varchar(6),
	product_id		varchar(6),
	quantity		int,
	order_total		numeric,
	order_date		date,
	PRIMARY KEY (order_id),
	FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
	FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE return_reasons (   
	return_reason_id    varchar(2) NOT NULL,
    reason_description  varchar(50),
    PRIMARY KEY(return_reason_id)
);
			
CREATE TABLE return_methods (   
	return_method_id    varchar(2) NOT NULL,
    method_description  varchar(50),
    primary key(return_method_id)
);

CREATE TABLE returns (   
	return_id           varchar(6) NOT NULL,
    order_id            varchar(6),
	quantity_returned   int,
	return_amount       numeric,
	return_reason_id    varchar(2),
	return_method_id    varchar(2),
    PRIMARY KEY(return_id),
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(return_reason_id) REFERENCES return_reasons(return_reason_id),
    FOREIGN KEY(return_method_id) REFERENCES return_methods(return_method_id)
);
"""

cur.execute(createCMD)
conn.commit()

### Reading in our data

In [6]:
import pandas as pd
import csv

In [7]:
os.chdir('/Users/francescaamato/Dropbox/Columbia/Fall 2022/SQL/Final/Data')

departments = pd.DataFrame(pd.read_csv('departments.csv'))
employees = pd.DataFrame(pd.read_csv('employees.csv'))
managers = pd.DataFrame(pd.read_csv('managers.csv'))
customers = pd.DataFrame(pd.read_csv('customers.csv'))
interaction_techniques = pd.DataFrame(pd.read_csv('interaction_techniques.csv'))
interaction_reasons = pd.DataFrame(pd.read_csv('interaction_reasons.csv'))
interactions = pd.DataFrame(pd.read_csv('interactions.csv'))
shop_categories = pd.DataFrame(pd.read_csv('shop_categories.csv'))
shop_subcategories = pd.DataFrame(pd.read_csv('shop_subcategories.csv'))
shops = pd.DataFrame(pd.read_csv('shops.csv'))
products = pd.DataFrame(pd.read_csv('products.csv'))
website_views = pd.DataFrame(pd.read_csv('website_views.csv'))
orders = pd.DataFrame(pd.read_csv('orders.csv'))
return_reasons = pd.DataFrame(pd.read_csv('return_reasons.csv'))
return_methods = pd.DataFrame(pd.read_csv('return_methods.csv'))
returns = pd.DataFrame(pd.read_csv('returns.csv'))

Setting our date columns:

In [9]:
shops['creation_date'] = pd.to_datetime(shops['creation_date'])
website_views['view_date'] = pd.to_datetime(website_views['view_date'])
website_views['view_time'] = pd.to_datetime(website_views['view_time'], format='%M:%S:%f')

Setting URLs to strings:

In [10]:
products['Product_URL'] = products['Product_URL'].astype(str)
products['Image_URL'] = products['Image_URL'].astype(str)

### Inserting our data into the database

In [11]:
#departments
for row in departments.itertuples():
    cur.execute('''
                INSERT INTO departments (id, department_name, annual_budget) 
                VALUES (%s, %s, %s)
                ''',
               (row.id, row.department_name, row.annual_budget))
conn.commit()

In [12]:
#employees
for row in employees.itertuples():
    cur.execute('''
                INSERT INTO employees (id, department_id, first_name, last_name, email, gender)
                VALUES (%s, %s, %s, %s, %s, %s)
                ''',
               (row.id, row.department_id, row.first_name, row.last_name, row.email, row.gender))
conn.commit()

In [13]:
#managers
for row in managers.itertuples():
    cur.execute('''
                INSERT INTO managers (manager_id, employee_id, department_id)
                VALUES (%s, %s, %s)
                ''',
               (row.manager_id, row.employee_id, row.department_id))
conn.commit()

In [14]:
#customers
for row in customers.itertuples():
    cur.execute('''
                INSERT INTO customers (customer_id, first_name, last_name, email, gender, address, state)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ''',
               (row.customer_id, row.first_name, row.last_name, row.email, row.gender, row.address, row.state))
conn.commit()

In [15]:
#interaction techniques
for row in interaction_techniques.itertuples():
    cur.execute('''
                INSERT INTO interaction_techniques (technique_id, description)
                VALUES (%s, %s)
                ''',
               (row.technique_id, row.description))
conn.commit()

In [16]:
#interaction reasons
for row in interaction_reasons.itertuples():
    cur.execute('''
                INSERT INTO interaction_reasons (reason_id, description)
                VALUES (%s, %s)
                ''',
               (row.reason_id, row.description))
conn.commit()

In [17]:
#interactions
for row in interactions.itertuples():
    cur.execute('''
                INSERT INTO interactions (interaction_id, customer_id, employee_id, technique_id, reason_id)
                VALUES (%s, %s, %s, %s, %s)
                ''',
               (row.interaction_id, row.customer_id, row.employee_id, row.technique_id, row.reason_id))
conn.commit()

In [18]:
#shop categories
for row in shop_categories.itertuples():
    cur.execute('''
                INSERT INTO shop_categories (category_id, category_description)
                VALUES (%s, %s)
                ''',
               (row.category_id, row.category_description))
conn.commit()

In [19]:
#shop subcategories
for row in shop_subcategories.itertuples():
    cur.execute('''
                INSERT INTO shop_subcategories (subcategory_id, subcategory_description)
                VALUES (%s, %s)
                ''',
               (row.subcategory_id, row.subcategory_description))
conn.commit()

In [20]:
#shops
for row in shops.itertuples():
    cur.execute('''
                INSERT INTO shops (shop_id, shop_name, category_id, subcategory_id, creation_date, listings_active_count, num_favorites, currency_code, is_shop_US_based, sale_message, sales_count, review_count, shop_location)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ''',
               (row.shop_id, row.shop_name, row.category_id, row.subcategory_id, row.creation_date, row.listing_active_count, row.num_favorers, row.currency_code, row.is_shop_us_based, row.sale_message, row.sales_count, row.review_count, row.shop_location))
conn.commit()

In [21]:
#products
for row in products.itertuples():
    cur.execute('''
                INSERT INTO products (product_id, product_name, seller_id, rating, reviews, price, badget, product_url, image_url)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ''',
               (row.product_id, row.product_name, row.seller_id, row.rating, row.reviews, row.price, row.badget, row.Product_URL, row.Image_URL))
conn.commit()

In [22]:
#website views
for row in website_views.itertuples():
    cur.execute('''
                INSERT INTO website_views (view_id, product_id, view_date, view_time, ip_address, total_seconds)
                VALUES (%s, %s, %s, %s, %s, %s)
                ''',
               (row.view_id, row.product_id, row.view_date, row.view_time, row.ip_address, row.total_seconds))
conn.commit()

In [23]:
#orders
for row in orders.itertuples():
    cur.execute('''
                INSERT INTO orders (order_id, customer_id, product_id, quantity, order_total, order_date)
                VALUES (%s, %s, %s, %s, %s, %s)
                ''',
               (row.order_id, row.customer_id, row.product_id, row.quantity, row.order_total, row.order_date))
conn.commit()

In [24]:
#return reasons
for row in return_reasons.itertuples():
    cur.execute('''
                INSERT INTO return_reasons (return_reason_id, reason_description)
                VALUES (%s, %s)
                ''',
               (row.return_reason_id, row.reason_description))
conn.commit()

In [25]:
#return methods
for row in return_methods.itertuples():
    cur.execute('''
                INSERT INTO return_methods (return_method_id, method_description)
                VALUES (%s, %s)
                ''',
               (row.return_method_id, row.method_description))
conn.commit()

In [26]:
#returns
for row in returns.itertuples():
    cur.execute('''
                INSERT INTO returns (return_id, order_id, quantity_returned, return_amount, return_reason_id, return_method_id)
                VALUES (%s, %s, %s, %s, %s, %s)
                ''',
                (row.return_id, row.order_id, row.quantity_returned, row.return_amount, row.return_reason_id, row.return_method_id))
conn.commit()